# Obsidian Geochemistry – Anillo Sourcing Notebook (Archival Version)

**Last meaningful update:** January 2026  
**Goal:** Long-term reproducible provenance analysis of obsidian using Rb–Sr–Zr (and potentially other traces)  
**Binder link:** [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/arf-berkeley/obsidian-source-anillo/main?labpath=notebooks/obsidian-geochem.ipynb)

### Repository structure expected
```
obsidian-source-anillo/
├── .binder/
│   └── requirements.txt
├── data/
│   ├── KRA21_sources.csv
│   ├── source_coords.csv       (optional – not used in this version)
│   └── study_samples.csv
├── notebooks/
│   └── obsidian-geochem.ipynb
└── figures/                   (gitignored or included)
```

Data is embedded → no external services → should still run in 2030+ (assuming Python 3.10–3.13 ecosystem)

## 1. Dependencies & Styling

Recommended `.binder/requirements.txt` content (pin these versions for reproducibility):
```
numpy==1.26.4
pandas==2.2.3
matplotlib==3.8.4
seaborn==0.13.2
scipy==1.13.1
scikit-learn==1.5.2
pyrolite==0.4.2
ipywidgets==8.1.5
jupyterlab-widgets==3.0.13
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import mahalanobis
from sklearn.covariance import EmpiricalCovariance
import pyrolite.plot
from pyrolite.util.plot import pyroplot_defaults
from pyrolite.transform import clr
import ipywidgets as widgets
from ipywidgets import interact, Output
from IPython.display import display, clear_output
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Nice defaults for pyrolite ternary etc.
pyroplot_defaults()

## 2. Load embedded data

In [ ]:
DATA_DIR = Path("../data")

def load_csv(name):
    path = DATA_DIR / f"{name}.csv"
    if not path.exists():
        raise FileNotFoundError(f"Missing: {path.resolve()}\nDid you commit the data files?")
    return pd.read_csv(path)

srcs  = load_csv("KRA21_sources")
study = load_csv("study_samples")

# Quick view
print(f"Sources: {len(srcs)} samples, {srcs['Group'].nunique()} groups")
print(f"Study:   {len(study)} samples")

display(srcs.head(3))
display(study.head(3))

## 3. Data QC & Schema check

In [ ]:
KEY_ELEMENTS = ["Rb", "Sr", "Zr"]           # core trio
STRING_COLS   = ["Sample", "Group"]

def enforce_schema(df, name):
    missing = set(KEY_ELEMENTS + STRING_COLS) - set(df.columns)
    if missing:
        raise ValueError(f"{name} missing columns: {missing}")
    df[STRING_COLS] = df[STRING_COLS].astype("string")
    df[KEY_ELEMENTS] = df[KEY_ELEMENTS].apply(pd.to_numeric, errors="coerce")
    print(f"{name} – missing values (%):\n", (df[KEY_ELEMENTS].isna().mean() * 100).round(1))

enforce_schema(srcs,  "Sources")
enforce_schema(study, "Study  ")

## 4. Normalization helper (compositional data)

In [ ]:
def normalize_composition(df, elements):
    """Row-wise sum to 1 (closed composition for ternary etc.)"""
    sub = df[elements].copy()
    row_sums = sub.sum(axis=1, skipna=False)
    return sub.div(row_sums, axis=0)

## 5. Interactive visualization (ipywidgets)

In [ ]:
group_options = sorted(srcs["Group"].dropna().unique())

group_selector = widgets.SelectMultiple(
    options=group_options,
    value=group_options[:min(6, len(group_options))],
    description="Groups:",
    rows=8,
    layout={"width": "max-content"}
)

output = Output()

def make_plots(selected_groups):
    with output:
        clear_output(wait=True)
        if not selected_groups:
            print("Select at least one group.")
            return

        filt = srcs[srcs["Group"].isin(selected_groups)]
        if len(filt) == 0:
            print("No samples in selected groups.")
            return

        norm_cols = KEY_ELEMENTS
        srcs_norm = normalize_composition(filt, norm_cols)
        study_norm = normalize_composition(study, norm_cols)

        # ─── Ternary + density ───────────────────────────────────────
        fig, ax = plt.subplots(figsize=(9, 8))
        pyrolite.plot.ternary_scatter(
            srcs_norm, components=norm_cols, ax=ax,
            c="C0", s=60, alpha=0.7, label="Sources"
        )
        pyrolite.plot.ternary_scatter(
            study_norm, components=norm_cols, ax=ax,
            marker="*", s=180, c="C3", label="Study samples"
        )
        pyrolite.plot.ternary_density_contour(
            srcs_norm, components=norm_cols, ax=ax,
            contours=[0.2, 0.5, 1.2, 3, 8], cmap="viridis", linewidths=0.9
        )
        ax.legend()
        ax.set_title(f"Rb–Sr–Zr Ternary – {len(selected_groups)} groups selected")
        plt.show()

        # ─── Biplot example (Rb vs Zr) ───────────────────────────────
        plt.figure(figsize=(8, 6))
        sns.scatterplot(data=filt, x="Rb", y="Zr", hue="Group", s=80, alpha=0.8)
        sns.scatterplot(data=study, x="Rb", y="Zr", color="black", marker="*", s=180, label="Study")
        plt.title("Rb vs Zr")
        plt.show()

interact(make_plots, selected_groups=group_selector);
display(group_selector, output)

## 6. Quantitative assignment (Mahalanobis distance)

In [ ]:
# Build group models (only groups with ≥ 3 samples)
group_models = {}
norm_cols = KEY_ELEMENTS

for g, sub in srcs.groupby("Group"):
    if len(sub) < 3:
        continue
    X = normalize_composition(sub, norm_cols)
    X_clr = clr(X.values)
    cov = EmpiricalCovariance().fit(X_clr)
    group_models[g] = {
        "mean": X_clr.mean(axis=0),
        "inv_cov": cov.precision_
    }

print(f"Usable source groups for Mahalanobis: {len(group_models)}")

# Assign study samples
results = []
study_norm = normalize_composition(study, norm_cols)
study_clr  = clr(study_norm.values)

for i, row in study.iterrows():
    dists = {}
    for g, m in group_models.items():
        d = mahalanobis(study_clr[i], m["mean"], m["inv_cov"])
        dists[g] = d
    if dists:
        best_g = min(dists, key=dists.get)
        best_d = dists[best_g]
    else:
        best_g, best_d = np.nan, np.nan
    results.append({"Sample": row["Sample"], "Best_Group": best_g, "Mahal_dist": best_d})

assignment_df = pd.DataFrame(results)
display(assignment_df.sort_values("Mahal_dist").head(12))
assignment_df.to_csv(DATA_DIR / "study_assignments.csv", index=False)
print("→ Saved to ../data/study_assignments.csv")

## 7. Final notes

- Add more elements to `KEY_ELEMENTS` when you have them (Nb, Y, Fe₂O₃ etc.)
- Consider saving static figures:
  ```python
  plt.savefig("../figures/ternary_selected.png", dpi=180, bbox_inches="tight")
  ```
- For true archival permanence → upload repo to Zenodo / Figshare → get DOI